# Zero2API Client

## Overview


Our purpose here is to illustrate data access (query) against an openly published table: 26 baboons 
with position fixes (in meters) every two seconds for a day, approximately. We have two forms of 
access: A basic table interface and a 'derived product' interface. The latter calculates the relative
dispersion of the baboon troop. 


The basic table-based API is used first. We'll call it `baboon1`. It supports queries over the four
table columns: time range, x-coordinate, y-coordinate (so a spatial bounding box) and individual identifier. 
We ignore the bounding box for this demonstration.


The second ('derived product') API was built separately from the first. In fact it makes use of the first 
`baboon1` interface to get its data. This is what we call *API composition*: Rather than make the basic 
API more complicated -- for example as we imagine new interesting calculations -- we push or compartmentalize 
the new calculation into other APIs.


### First (basic) API examples

In [ ]:
# libraries used for talking with the two APIs
import pandas as pd
import requests

In [43]:
# use the 'baboons1' interface to get 11 rows of data

indiv = 10              # there are at least 38 GPS collars; we have here data for only 26 of them 
table = 'false'         # tabular data flag false means 'send the result as JSON'
t0 = '0:02:52'          # Time range begins at 2 minutes 52 seconds; duration 32 seconds
t1 = '0:03:24'          #   but only 22 seconds of data are returned

baseURL = 'https://rlu4ch9a57.execute-api.us-west-2.amazonaws.com/default/baboon1'
url     = baseURL + '?indiv={}&table={}&t0={}&t1={}'.format(indiv,table,t0,t1)

a=pd.read_json(url)     # the url contains the details of the query; issued here
a                       # the response is a pandas DataFrame with 11 rows of data

,indiv,time,x,y
0,10,0:03:04,-778.263144,428.061810
1,10,0:03:06,-778.207484,428.095205
2,10,0:03:08,-777.884659,428.351235
3,10,0:03:10,-777.795603,428.395762
4,10,0:03:12,-777.328063,428.362367
5,10,0:03:14,-777.072029,428.262181
6,10,0:03:16,-776.059025,428.707449
7,10,0:03:18,-775.112812,429.008006
8,10,0:03:20,-774.500557,429.130454
9,10,0:03:22,-773.220387,429.798357


In [44]:
# This illustrates some pandas DataFrame attributes

print(type(a), '\n\n', a, '\n\n', a.index, '\n\n', a.columns, '\n\n', a['time'], '\n\n', a['x'][0:3])

<class 'pandas.core.frame.DataFrame'> 

     indiv     time           x           y
0      10  0:03:04 -778.263144  428.061810
1      10  0:03:06 -778.207484  428.095205
2      10  0:03:08 -777.884659  428.351235
3      10  0:03:10 -777.795603  428.395762
4      10  0:03:12 -777.328063  428.362367
5      10  0:03:14 -777.072029  428.262181
6      10  0:03:16 -776.059025  428.707449
7      10  0:03:18 -775.112812  429.008006
8      10  0:03:20 -774.500557  429.130454
9      10  0:03:22 -773.220387  429.798357
10     10  0:03:24 -773.220387  429.653645 

 RangeIndex(start=0, stop=11, step=1) 

 Index(['indiv', 'time', 'x', 'y'], dtype='object') 

 0     0:03:04
1     0:03:06
2     0:03:08
3     0:03:10
4     0:03:12
5     0:03:14
6     0:03:16
7     0:03:18
8     0:03:20
9     0:03:22
10    0:03:24
Name: time, dtype: object 

 0   -778.263144
1   -778.207484
2   -777.884659
Name: x, dtype: float64


In [45]:
# test cell: try various types of query including bounding box (not working yet)

indiv = 10
table = 'false'
t0 = '0:02:52'
t1 = '0:03:24'
x0 = -770.
x1 = -780.
y0 =  430.
y1 =  420.

# full scan
# url = baseURL + '?indiv={}&table={}&t0={}&t1={}&x0={}&x1={}&y0={}&y1={}'.format(indiv,table,t0,t1,x0,x1,y1,y0)

# scan sans indiv
# url = baseURL + '?table={}&t0={}&t1={}&x0={}&x1={}&y0={}&y1={}'.format(table,t0,t1,x0,x1,y1,y0)

# query indiv and time: Working
# url = baseURL + '?indiv={}&table={}&t0={}&t1={}'.format(indiv,table,t0,t1)

# url = baseURL + '?table={}&t0={}&t1={}'.format(table,t0,t1)
# url = baseURL + '?indiv={}&table={}'.format(indiv,table)

# a=pd.read_json(url)
# print(url, '\n\n', a)

### Second (moment of inertia) API example

In [46]:
# obtain one moment of inertia value for the entire troupe using one minute of data

urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
t0 = '11:00:00'
t1 = '11:01:00'
url= urlcomp + '?moi=true&t0={}&t1={}'.format(t0,t1)
a = requests.get(url)
print(a.content.decode('utf-8'))

moi is 10.30867843517405 from 775 locations


In [47]:
# test a bad query (the times t0 and t1 are not usable)
#   this should produce 'zero' or some sort of fail signal

urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
t0 = 'abcd'
t1 = 'efgh'
url= urlcomp + '?moi=true&t0={}&t1={}'.format(t0,t1)
a = requests.get(url)
print(a.content.decode('utf-8'))

moi is 0.0 from 0 locations


In [48]:
# Loop over one hour in three-minute intervals to produce 20 moments of inertia

urlcomp = 'https://ycg3xmhjwj.execute-api.us-west-2.amazonaws.com/default/babooncomposition'
b=[]
for i in range(20):
    mymin0 = i * 3
    mymin1 = mymin0 + 3
    if mymin0 < 10: mymin0string = '0' + str(mymin0)
    else:           mymin0string = str(mymin0)
    if mymin1 < 10: mymin1string = '0' + str(mymin1)
    else:           mymin1string = str(mymin1)
    t0 = '4:' + mymin0string + ':00'
    t1 = '4:' + mymin1string + ':00'
    if t1 == '4:60:00': t1 = '5:00:00'
    url= urlcomp + '?moi=true&t0={}&t1={}'.format(t0,t1)
    a = requests.get(url)
    b.append(float(a.content.decode("utf-8").split(' ')[2]))

print(b)

[47.24668210047696, 47.71861072775954, 48.92386733276165, 49.34812811013475, 48.8155080002324, 47.06645781119527, 49.71762649918903, 70.1851986629111, 84.94120700100285, 78.46260152031512, 66.99084135228593, 74.55004855452188, 86.66621643361388, 92.59470487945826, 90.57328670073204, 74.01396790676765, 55.93187417612399, 32.35153076898543, 20.589211909522685, 16.512057140423554]


In [32]:
# write as text file the accumulated moi values

with open('hour4_3min_moi_values.txt', 'w') as f:
    for item in b: f.write("%s\n" % item)